In [1]:
import pandas as pd
import numpy as np
import csv
import MeCab
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import re
import neologdn
import demoji
import emoji

model_dir = '/Users/iomacbookair2/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
# model_dir = '/Users/labimac/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
model_word2vec = KeyedVectors.load_word2vec_format(model_dir, binary=True)

In [2]:
def calc_similarity_word2vec(sentence1, sentence2, model):
  mecab = MeCab.Tagger(
      '-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
  pre_sentence1 = sentence1.split(" ")
  sentence1_words = []
  for s1 in pre_sentence1:
    sentence1_words += [line.split("\t")[0]
                        for line in mecab.parse(s1).split("\n")[:-2]]
  pre_sentence2 = sentence2.split(" ")
  sentence2_words = []
  for s2 in pre_sentence2:
    sentence2_words += [line.split("\t")[0]
                        for line in mecab.parse(s2).split("\n")[:-2]]
  if not sentence1_words and not sentence2_words:
    similarity = 1
  elif not sentence1_words or not sentence2_words:
    similarity = 0
  elif (len(sentence1_words) == 1 and len(sentence2_words) != 1) or (len(sentence2_words) == 1 and len(sentence1_words) != 1):
    similarity = 0
  elif all(re.match(r'.*[a-zA-Z].*', word) for word in sentence1_words) or all(re.match(r'.*[a-zA-Z].*', word) for word in sentence2_words):
    similarity = 0
  else:
    sentence1_embedding = np.mean([model[word]
                                  for word in sentence1_words if word in model], axis=0)
    sentence2_embedding = np.mean([model[word]
                                  for word in sentence2_words if word in model], axis=0)
    if np.isnan(sentence1_embedding).any() or np.isnan(sentence2_embedding).any():
        similarity = 0
    else:
      similarity = cosine_similarity(
          [sentence1_embedding], [sentence2_embedding])[0][0]
  return similarity


def calc_average_similarity_word2vec(sentences, key):
    similarities = []
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            similarity = calc_similarity_word2vec(
                sentences[i], sentences[j], model_word2vec)
            similarities.append(similarity)
    q = [0, 0.25, 0.5, 0.75, 1]
    outputs = {}
    for i in range(len(q)):
        outputs[f"quantile_{i}"] = np.quantile(similarities, q[i])
    outputs["q1"] = outputs["quantile_1"]
    outputs["q2"] = outputs["quantile_2"]
    outputs["q3"] = outputs["quantile_3"]
    outputs["standard_deviation"] = np.std(similarities)
    outputs["mean"] = np.mean(similarities)
    if key == "q2":
        return outputs["quantile_2"]
    elif key == "q1":
        return outputs["quantile_1"]
    elif key == "q3":
        return outputs["quantile_3"]
    elif key == "standard_deviation":
        return outputs["standard_deviation"]
    elif key == "mean":
        return outputs["mean"]


In [3]:
path = "/Users/iomacbookair2/Documents/lab/DEIM2023/tweet_csv/nichiten/230115_nichiten.csv"
# path = "/Users/labimac/Documents/lab/DEIM2023/tweet_csv/221214_ann_wed.csv"
df = pd.read_csv((path))
df.sort_values(by = 'created_at', ascending = True, inplace = True)
df = df.reset_index(drop=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.drop("author_id", axis=1)
df = df.drop("username", axis=1)
df = df.drop("tweet_id", axis=1)
df = df.drop("like_count", axis=1)
df = df.drop("retweet_count", axis=1)

df.to_csv("sorted.csv", index=False)
df

,created_at,text
0,2023-01-15 10:00:00,#nichiten おはようございます📻
1,2023-01-15 10:00:01,安住さん、中澤さん、スタッフのみなさん、おはようございます。 そして、安住さんおかえりなさい...
2,2023-01-15 10:00:01,#nichiten
3,2023-01-15 10:00:01,#nichiten #安住紳一郎 さぁ！ ”nichiten安住”の2023年 始まりで...
4,2023-01-15 10:00:01,【局長待遇大復活祭】おはようございます。 #nichiten
...,...,...
3934,2023-01-15 11:59:10,エンディングは 「show me」 コロナに罹った安住さんが コロナをないがしろにする 政府...
3935,2023-01-15 11:59:13,安住さん🥸中澤さん👸 高橋洋子さん👸 今日も楽しい2時間でした！ 安住さん🥸、 お身体をお大...
3936,2023-01-15 11:59:33,中尾明慶さん、好きな俳優さん😆#nichiten
3937,2023-01-15 11:59:44,黒木華ちゃんの『凪のお暇』も『重版出来！』も良かったなぁ。 #nichiten


In [4]:
def preprocess(text):
    text = emoji.replace_emoji(text, replace=' ')
    text = neologdn.normalize(text)
    text = re.sub(r'#\S+', '', text) # ハッシュタグをスペースに置き換え
    text = re.sub(r'＃\S+', '', text) # ハッシュタグをスペースに置き換え
    text = re.sub(r'http?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text) # URLをスペースに置き換え
    text = re.sub(r'https?://\S+', ' ', text) # URLをスペースに置き換え
    text = re.sub(r'[!-/:-@[-`{-~]', r' ', text) # 記号をスペースに置き換え
    text = re.sub(
        u'[■-♯【】「」『』・ㅂﾟˊᗜ、。∀〇╰ˋω…╭´｀•˘д↑艸╯→°д̀ᴗ˃˂⁽⁾φl└＼※彡𖥦←ꂹ]', ' ', text) # 記号をスペースに置き換え
    text = re.sub(r'(\d)([,.])(\d+)', r'\1\3', text) # 小数点とカンマを消す
    text = re.sub(r'\d+', '0', text) # 数字を0に置き換え
    text = text.lower() # 英字を小文字に
    text = re.sub(r"[\u3000\t\r\n]", " ", text) # 空白文字をスペースに置き換え
    return text


df['text'] = df['text'].apply(preprocess)
df
df.to_csv('preprocessed.csv', index=False)


In [5]:
groups = df.groupby(pd.Grouper(key='created_at', freq='min'))
df_texts_by_minute = pd.DataFrame({
    "texts_by_minute": groups.apply(lambda x: x["text"].tolist())
})
df_texts_by_minute = df_texts_by_minute.reset_index()
df_texts_by_minute

,created_at,texts_by_minute
0,2023-01-15 10:00:00,"[, 安住さん 中澤さん スタッフのみなさん おはようございます そして 安住さんおかえりな..."
1,2023-01-15 10:01:00,"[安住紳一郎の日曜天国はじまた , おはようございます , おはようございます 昨日のnキ..."
2,2023-01-15 10:02:00,"[安住さん まだ声が心配してだね , おはよう御座います安住さんお帰りなさい , オハヨウゴ..."
3,2023-01-15 10:03:00,"[おはようございます 安住アナは お帰りなさい やっぱり 声がまだ本調子じゃないね , ..."
4,2023-01-15 10:04:00,"[まだやんのかい 笑 , まだ鼻声だね 喉をお大事に , 白い巨塔は唐沢寿明の平成生まれで..."
...,...,...
115,2023-01-15 11:55:00,"[うほほ なつかしい曲 , そうか 冷凍和風野菜でカレー作ればいいんだ , 最後は東野英治..."
116,2023-01-15 11:56:00,"[実況おつでした , show me 懐かしい, あなたらしくもないは, ありがとうござ..."
117,2023-01-15 11:57:00,"[あっという間に終わった感 楽しかった , 楽しかったです 素晴らしい放送に拍手 安住さん..."
118,2023-01-15 11:58:00,"[安住さんお大事に喉は無理すると長引くからご無理なさらず, シナモンは期限切れてても問題なく..."


In [6]:
df_texts_by_minute["q1"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "q1"))


/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/

In [7]:
df_texts_by_minute["q2"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "q2"))


/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/

In [8]:
df_texts_by_minute["q3"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "q3"))


/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/

In [9]:
df_texts_by_minute["stdev"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "standard_deviation"))


/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/iomacbookair2/miniforge3/envs/lab/lib/python3.9/

In [10]:
df_texts_by_minute

,created_at,texts_by_minute,q1,q2,q3,stdev
0,2023-01-15 10:00:00,"[, 安住さん 中澤さん スタッフのみなさん おはようございます そして 安住さんおかえりな...",0.000000,0.247872,0.733923,0.366441
1,2023-01-15 10:01:00,"[安住紳一郎の日曜天国はじまた , おはようございます , おはようございます 昨日のnキ...",0.165027,0.453224,0.658940,0.301978
2,2023-01-15 10:02:00,"[安住さん まだ声が心配してだね , おはよう御座います安住さんお帰りなさい , オハヨウゴ...",0.237295,0.479636,0.636866,0.263714
3,2023-01-15 10:03:00,"[おはようございます 安住アナは お帰りなさい やっぱり 声がまだ本調子じゃないね , ...",0.048211,0.438705,0.661849,0.301990
4,2023-01-15 10:04:00,"[まだやんのかい 笑 , まだ鼻声だね 喉をお大事に , 白い巨塔は唐沢寿明の平成生まれで...",0.216755,0.484878,0.684719,0.276789
...,...,...,...,...,...,...
115,2023-01-15 11:55:00,"[うほほ なつかしい曲 , そうか 冷凍和風野菜でカレー作ればいいんだ , 最後は東野英治...",0.329736,0.571705,0.714191,0.242108
116,2023-01-15 11:56:00,"[実況おつでした , show me 懐かしい, あなたらしくもないは, ありがとうござ...",0.397188,0.610091,0.730461,0.237783
117,2023-01-15 11:57:00,"[あっという間に終わった感 楽しかった , 楽しかったです 素晴らしい放送に拍手 安住さん...",0.446219,0.674674,0.798473,0.324671
118,2023-01-15 11:58:00,"[安住さんお大事に喉は無理すると長引くからご無理なさらず, シナモンは期限切れてても問題なく...",0.318259,0.770827,0.836587,0.351700


In [11]:
df_texts_by_minute.to_csv('220115_similarity.csv', index=False)